# Lesson 2.3: DateTime strings, null functions & logical order of processing queries

### Lesson Duration: 3 hours

> Purpose: The purpose of this lesson is to teach students how to use other advanced functions such as _DateTime_ functions and _null_ functions for data processing and filtering. Also, acknowledging and understanding the logical order of processing queries will help the students to write efficient queries and debug errors.

---

### Setup

To start this lesson, students should have:

- Completed lesson 2.2
- All previous Setup

---

### Learning Objectives

After this lesson, students will be able to:

- Manipulate DateTime data to suit your projects requirements
- Apply multiple conditions using the `WHERE` clause and `case` statement

---

### Lesson 1 key concepts

> :clock10: 20 min

Using DateTime functions

- Converting other data types to date, DateTime
- Changing formats of date columns using `date_format()`

> :exclamation: In this session, go through the case study pdf document to understand the format in which dates have been recorded in these tables - `account`, `card`, and `loan`.

<details>
<summary> Click for Code Sample </summary>

:exclamation: Keep working on the `bank` database.

In the first table, the column `date` is of type integer. So we will convert the column into date format.
For now, this change will only be temporary as we not altering the structure of the table where the column has been defined.

```sql
select account_id, district_id, frequency, convert(date,date) from bank.account;
```

In the function, `convert()`, the first argument is the name of the column and the second is the type to which you want to convert. Similarly, we can do it for the `loan` table:
`select CONVERT(date,date) from bank.loan;`.

```sql
select account_id, district_id, frequency, CONVERT(date,datetime) from bank.account;

-- next is a two step process:
select substring_index(issued, ' ', 1) from bank.card;
select convert(SUBSTRING_INDEX(issued, ' ', 1), date) from bank.card;
```

A list of formats can be found [here](https://www.w3schools.com/sql/func_mysql_date_format.asp).

```sql
-- converting the original format to the date format that we need:
select date_format(convert(date,date), '%Y-%M-%D') from bank.loan;

-- if we just want to extract some specific part of the date
select date_format(convert(date,date), '%Y') from bank.loan;
```

# 2.03 Activity 1

Keep working on the `bank` database. (_In case you need to load data again, refer to the previous lab and `files_for_lab` folder to get the database._)

#### Queries

1. Get `card_id` and `year_issued` for all gold cards.
2. When was the first gold card **issued**? (Year)
3. Get issue date as:

   - `date_issued`: 'November 7th, 1993'
   - `fecha_emision`: '07 of November of 1993'

### Solutions:

### 1

```sql
select card_id, date_format(convert(SUBSTRING_INDEX(issued, ' ', 1), date), '%Y') as year_issued
from bank.card
where type = 'gold'
limit 10;
```

### 2

```sql
select min(date_format(convert(SUBSTRING_INDEX(issued, ' ', 1), date), '%Y')) as year_issued
from bank.card
where type = 'gold';
```

### 3

```sql
select date_format(convert(SUBSTRING_INDEX(issued, ' ', 1), date), '%M %D, %Y') as year_issued,
       date_format(convert(SUBSTRING_INDEX(issued, ' ', 1), date), '%d of %M of %Y') as fecha_emision
from bank.card
limit 10;
```

### Lesson 2 key concepts

> :clock10: 20 min

- Logical order of processing SQL queries

> In this section, we will discuss the logical order of query processing in SQL. Even though if someone would read the SQL query, it is likely that they would think that the query is processed line after line from the beginning. But the way SQL reads the query is different. Here we will explain that order using a couple of examples. This is important to understand as this would help them write better, more efficient queries later, and will also help them debug the errors. The order is shown below:

1.  FROM
2.  ON
3.  JOIN
4.  WHERE
5.  GROUP BY
6.  WITH CUBE/ROLLUP
7.  HAVING
8.  SELECT
9.  DISTINCT
10. ORDER BY
11. TOP/LIMIT
12. OFFSET/FETCH

> Here is a [link](https://www.itprotoday.com/sql-server/logical-query-processing-what-it-and-what-it-means-you) to the logical order of processing SQL queries.

<details>
<summary> Click for Code Sample: Logical order of processing </summary>

```sql
select * from bank.card
where type = 'classic'
order by card_id
limit 10;

select * from bank.order
where k_symbol = 'SIPO' and amount > 5000
order by order_id desc
limit 10;
```

### Lesson 3 key concepts

> :clock10: 20 min

Null functions

- What a `null` value means in SQL - Three valued logic in SQL
- Checking for the `null` values

:exclamation: Please refer to the [document](files_for_lesson_and_activities//null_values_three_value_logic.md) in this lesson's folder.

<details>
<summary> Click for Code Sample: Null functions </summary>

```sql
select isnull('Hello');
select isnull(card_id) from bank.card;

-- this is used to check all the elements of a column.
-- 0 means not null, 1 means null
select sum(isnull(card_id)) from bank.card;

select * from bank.order
where k_symbol is null;
```

As you might have noticed in this case, even though we see a lot of missing values in the column `k_symbol`, the above query does not filter those rows. It might be because those columns actually have value, for example, empty space. SQL considers that as a character/ value as well. So we will check for that now:

```sql
select * from bank.order
where k_symbol is not null and k_symbol = ' ';
```

# 2.03 Activity 3

#### Questions and queries

1. Null or empty values are usually a problem. Think about why those null values can appear and think of ways of dealing with them.
2. Check for _transactions_ with null or empty values for the column `amount`.
3. Count how many _transactions_ have empty and non-empty `k_symbol` (in one query).

### Solutions:

### 1

- Missing data
- Sensor errors
- New columns
- Replace with a placeholder value

### 2

```sql
select * from bank.trans
where amount is null;
```

### 3

```sql
select sum(k_symbol = ' ') as k_symbol_empty, 
   sum(not k_symbol = ' ') as k_symbol_non_empty
from bank.trans
where amount is not null;
```

### Lesson 4 key concepts

> :clock10: 20 min

Case statements in SQL

- Why do we need case statements

      - Using case statements

- Data types in SQL (will be added to the next session)

<details>
<summary> Click for Code Sample </summary>

In the `loan` table, there's column status A, B, C, and D. Using the case statement we will try to replace the values there with a brief description.

```sql
select loan_id, account_id,
case
when status = 'A' then 'Good - Contract Finished'
when status = 'B' then 'Defaulter - Contract Finished'
when status = 'C' then 'Good - Contract Running'
else 'In Debt - Contract Running'
end as 'Status_Description'
from bank.loan;
```

# Lab | SQL Queries 3

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals. You have been using this database for a couple labs already, but if you need to get the data again, refer to the official [installation link](https://dev.mysql.com/doc/sakila/en/sakila-installation.html).

The database is structured as follows:
![DB schema](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/database-sakila-schema.png)

<br><br>

### Instructions

1. How many distinct (different) actors' last names are there?
2. In how many different languages where the films originally produced? (Use the column `language_id` from the `film` table)
3. How many movies were released with `"PG-13"` rating?
4. Get 10 the longest movies from 2006.
5. How many days has been the company operating (check `DATEDIFF()` function)?
6. Show rental info with additional columns month and weekday. Get 20.
7. Add an additional column `day_type` with values 'weekend' and 'workday' depending on the rental day of the week.
8. How many rentals were in the last month of activity?

### Lab Solutions:

-- 1

```sql
select count(distinct last_name)
from actor;
```


-- 2

```sql
select count(distinct language_id)
from film;
```

-- 3

```sql
select count(*)
from film
where rating = "PG-13";
```

-- 4

```sql
select title, length
from film
where release_year = 2006
order by length desc
limit 10;
```

-- 5

```sql
select datediff(max(rental_date), min(rental_date)) as active_days
from rental;
```

-- 6

```sql
select *, date_format(rental_date, '%M') as month , date_format(rental_date, '%W') as weekday
from rental
limit 20;
```

-- 7

```sql
select *, case when date_format(rental_date, '%W') in ('Saturday', 'Sunday')
          then 'weekend'
          else 'workday' end as day_type
from rental;
```

-- 8

```sql
select date(max(rental_date))- INTERVAL 30 DAY, date(max(rental_date))
from rental;

select count(*)
from rental
where date(rental_date) between date('2006-01-15') and date('2006-02-14')
```